# Auto Insurance Claim Fraud Detection

In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from tabulate import tabulate

## Data Preprocessing
We will begin by reading in and taking a quick look at our data.

In [3]:
raw_data = pd.read_csv(os.getcwd() + '\insurance_claims.csv')
raw_data.head()

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported
0,328,48,521585,2014-10-17,OH,250/500,1000,1406.91,0,466132,...,2,YES,71610,6510,13020,52080,Saab,92x,2004,Y
1,228,42,342868,2006-06-27,IN,250/500,2000,1197.22,5000000,468176,...,0,?,5070,780,780,3510,Mercedes,E400,2007,Y
2,134,29,687698,2000-09-06,OH,100/300,2000,1413.14,5000000,430632,...,3,NO,34650,7700,3850,23100,Dodge,RAM,2007,N
3,256,41,227811,1990-05-25,IL,250/500,2000,1415.74,6000000,608117,...,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y
4,228,44,367455,2014-06-06,IL,500/1000,1000,1583.91,6000000,610706,...,1,NO,6500,1300,650,4550,Accura,RSX,2009,N


Notice that the following columns contain entries with a "?" which denotes a missing value
    <br />&nbsp;&nbsp;&nbsp;&nbsp;• *collision_type*
    <br />&nbsp;&nbsp;&nbsp;&nbsp;• *property_damage*
    <br />&nbsp;&nbsp;&nbsp;&nbsp;• *police_report_available*
    <br />

We will replace these entries with NANs.
<br />
Furthermore, we will convert the '*fraud_reported*' column, which is our target variable, to binary.

In [4]:
raw_data.replace('?', np.nan, inplace = True)
raw_data['fraud_reported'] = raw_data['fraud_reported'].map({'Y': 1, 'N': 0})

Next, we identify the columns with missing values and their respective amount.
<br />

Looking below, these columns are:
<br />&nbsp;• *collision_type*
<br />&nbsp;• *property_damage*
<br />&nbsp;• *police_report_available*
<br />

In [5]:
raw_data.isnull().sum()[raw_data.isnull().sum() > 0]

collision_type             178
property_damage            360
police_report_available    343
dtype: int64

First, we take a look at the *collision_type* column.
<br />
Below we determine that the corresponding *incident_type* for all missing values of *collision_type* are either **Vehicle Theft** or **Parked Car**.
<br />
Furthermore, *collision_type* is missing for all rows where the *incident_type* is either **Vehicle Theft** or **Parked Car**.
<br />
In both instances, the policy holder is not present to witness the damage done to their car.
<br />
And so, we replace all NAN values for this column with 'Unknown'.

In [6]:
incident_type_where_collision_type_missing = raw_data.loc[raw_data['collision_type'].isna()]['incident_type'].unique()
print("Entries for incident_type where collision_type is missing: " + str(incident_type_where_collision_type_missing))
print("Entries for collision_type where incident_type is in " + str(incident_type_where_collision_type_missing) + ": " +
        str(raw_data.loc[raw_data['incident_type'].isin(incident_type_where_collision_type_missing)]['collision_type'].unique()))
raw_data['collision_type'].replace(np.nan, 'Unknown', inplace = True)

Entries for incident_type where collision_type is missing: ['Vehicle Theft' 'Parked Car']
Entries for collision_type where incident_type is in ['Vehicle Theft' 'Parked Car']: [nan]


Moving on to the remaining columns that contain missing values: *property_damage* and *police_report_available*.<br />
There doesn't seem to be a connection in the entries where *property_damage* or *police_report_available* data is missing.<br />
One would assume that if property damage and a police report were present, it would be less likely to be a case of fraud.<br />
However, these details simply being missing is not a good enough indicator of fraud.<br />
These do not seem like values that we can impute with accuracy, so we will also change these missing values to 'unknown'

In [7]:
raw_data['property_damage'].replace(np.nan, 'Unknown', inplace = True)
raw_data['police_report_available'].replace(np.nan, 'Unknown', inplace = True)

## Evaluating A Baseline Model
In this section we will create and evaluate a baseline model. <br />
Our baseline model will be a simple Decision Tree that is trained using only the numeric data from our dataset. <br />
Below we see that the accuracy score of our baseline model is 64.4%


In [8]:
# define the list of columns containing non-numeric values
non_numeric_cols = ['policy_csl', 'insured_sex', 'insured_education_level', 'insured_relationship', 'incident_type', 'collision_type',
                        'incident_severity', 'authorities_contacted', 'property_damage', 'police_report_available', 'policy_bind_date',
                        'insured_zip', 'insured_hobbies', 'incident_date', 'incident_city', 'incident_location', 'auto_make', 'auto_model',
                        'policy_state', 'insured_occupation', 'incident_state']

# create a subset of our data, dropping the non-numeric and target columns and divide into a 75/25 split
baseline_X = raw_data.copy()
baseline_X.drop(non_numeric_cols, inplace=True, axis=1)
baseline_X = baseline_X.drop('fraud_reported', axis=1)
y = raw_data['fraud_reported']
train_bl_X, val_bl_X, train_bl_y, val_bl_y = train_test_split(baseline_X, y, test_size=0.25, random_state=0)

# the baseline model is defined, fit and the accuracy score is recorded to be referenced later
baseline_model = DecisionTreeClassifier(random_state=0)
baseline_model.fit(train_bl_X, train_bl_y)
baseline_predictions = baseline_model.predict(val_bl_X)
baseline_score = accuracy_score(val_bl_y, baseline_predictions)
print('Baseline Model Accuracy: ' + str(round(baseline_score * 100, 2)) + "%")

Baseline Model Accuracy: 64.4%


## Feature Engineering
Attempting to improve from our baseline model, we begin by sorting the columns of interest into numeric and categorical columns. <br />
Taking a closer look at our categorical columns below, we see that they each have a reasonable number of unique values.

In [9]:
numerical_cols = ['months_as_customer', 'age', 'policy_deductable', 'policy_annual_premium', 'umbrella_limit', 'capital_gains', 'capital_loss',
                    'incident_hour_of_the_day', 'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'total_claim_amount', 'injury_claim',
                    'property_claim', 'vehicle_claim']
categorical_cols = ['policy_csl', 'insured_sex', 'insured_education_level', 'insured_relationship', 'incident_type', 'collision_type',
                        'incident_severity', 'authorities_contacted', 'property_damage', 'police_report_available']

# for each categorical column, output the column name and number of unique values
for col in categorical_cols:
    print(col + ": " + str(len(raw_data[col].unique())))

policy_csl: 3
insured_sex: 2
insured_education_level: 7
insured_relationship: 6
incident_type: 4
collision_type: 4
incident_severity: 4
authorities_contacted: 5
property_damage: 3
police_report_available: 3


Next, the categorical columns are converted into indicator variables using *pd.get_dummies*. <br />
We drop the first column to help reduce the extra columns created, thereby reducing the correlations created among dummy variables. <br />
All of the relevant data is then combined into X, which is now a DataFrame containing solely numeric data.

In [10]:
categorical_df = pd.get_dummies(raw_data[categorical_cols], drop_first=True)
numerical_df = raw_data[numerical_cols]
X = pd.concat([numerical_df, categorical_df], axis=1)
X.head()

,months_as_customer,age,policy_deductable,policy_annual_premium,umbrella_limit,capital_gains,capital_loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,...,incident_severity_Total Loss,incident_severity_Trivial Damage,authorities_contacted_Fire,authorities_contacted_None,authorities_contacted_Other,authorities_contacted_Police,property_damage_Unknown,property_damage_YES,police_report_available_Unknown,police_report_available_YES
0,328,48,1000,1406.91,0,53300,0,5,1,1,...,0,0,0,0,0,1,0,1,0,1
1,228,42,2000,1197.22,5000000,0,0,8,1,0,...,0,0,0,0,0,1,1,0,1,0
2,134,29,2000,1413.14,5000000,35100,0,7,3,2,...,0,0,0,0,0,1,0,0,0,0
3,256,41,2000,1415.74,6000000,48900,-62400,5,1,1,...,0,0,0,0,0,1,1,0,0,0
4,228,44,1000,1583.91,6000000,66000,-46000,20,1,0,...,0,0,0,1,0,0,0,0,0,0


In this section of feature engineering, we will focus on reducing any prevalent multicollinearity. <br />
We will use VIF scores to determine the variables with significant multicollinearity, which will have a score of 10 or greater. <br />

Taking a look at the scores below, we can deduce the following:
<br />&nbsp;&nbsp;&nbsp;&nbsp;• *age* is highly correlated to *months_as_customer*
<br />&nbsp;&nbsp;&nbsp;&nbsp;• *total_claim_amount*, *vehicle_claim*, *property_claim*, *injury_claim* are all highly correlated as *total_claim_amount* is a sum of the rest
<br />&nbsp;&nbsp;&nbsp;&nbsp;• *collision_type_unknown* is highly correlated to *incident_type_Vehicle Theft* and *incident_type_Parked Car* because the collison type is always unknown in those two incident types
<br />&nbsp;&nbsp;&nbsp;&nbsp;• *incident_type_Single Vehicle Collision* is highly correlated to *number_of_vehicles_involved*
<br />&nbsp;&nbsp;&nbsp;&nbsp;• *policy_annual_premium* and *vehicle_claim* are highly correlated to other variables because the are determined depending on the other variables


In [11]:
def get_VIF(df):
    vif_data = pd.DataFrame()
    vif_data["feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(len(df.columns))]
    return vif_data

vif_scores = get_VIF(X)
print(vif_scores[vif_scores["VIF"] >= 10].sort_values('VIF'))

c:\Users\Daniel\Anaconda3\envs\AutoInsFraud\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


                                   feature         VIF
30  incident_type_Single Vehicle Collision   14.564231
3                    policy_annual_premium   26.270979
0                       months_as_customer   26.583771
8              number_of_vehicles_involved   35.794012
1                                      age  117.837187
11                      total_claim_amount         inf
12                            injury_claim         inf
13                          property_claim         inf
14                           vehicle_claim         inf
29                incident_type_Parked Car         inf
31             incident_type_Vehicle Theft         inf
34                  collision_type_Unknown         inf


Based on the above deductions, the below columns are dropped:
<br />&nbsp;&nbsp;&nbsp;&nbsp;• *age*
<br />&nbsp;&nbsp;&nbsp;&nbsp;• *total_claim_amount*
<br />&nbsp;&nbsp;&nbsp;&nbsp;• *collision_type_unknown*
<br />&nbsp;&nbsp;&nbsp;&nbsp;• *incident_type_Single Vehicle Collision*
<br />&nbsp;&nbsp;&nbsp;&nbsp;• *policy_annual_premium*
<br />&nbsp;&nbsp;&nbsp;&nbsp;• *vehicle_claim*
<br />
<br />
Looking at the VIF scores again, there no longer remains a column with a score of at least 10. <br />
And so we have successfully reduced any prevalent multicollinearity.


In [12]:
X.drop(['age', 'total_claim_amount', 'collision_type_Unknown', 'incident_type_Single Vehicle Collision', 'policy_annual_premium', 'vehicle_claim'],
            inplace=True, axis=1)
vif_scores = get_VIF(X)
print(vif_scores[vif_scores["VIF"] >= 10].sort_values('VIF'))

Empty DataFrame
Columns: [feature, VIF]
Index: []


## Model Initialization
In this section, the base model for each machine learning algorithm and its associated parameters for tuning are initialized. <br />
We also define the number of folds for cross validation to be 5.<br />
This means that each time a model is trained, it is trained 5 times; each of which is on a shuffling 4/5ths of the data and then tested on the remaining 1/5th. <br />

In [13]:
# define the models and their associated parameters for hypertuning
dt_model = DecisionTreeClassifier(random_state=0)
rf_model = RandomForestClassifier(random_state=0)
gb_model = GradientBoostingClassifier(random_state=0)
xgb_model = XGBClassifier(random_state=0)
knn_model = KNeighborsClassifier()

dt_params = {'max_depth': [3, 5, 10, 15, 20],
             'min_samples_leaf': [5, 10, 20, 50, 100]}
rf_params = {'n_estimators': [10, 25, 50, 75, 100],
             'max_depth': [3, 5, 10, 15, 20],
             'min_samples_leaf': [5, 10, 20, 50, 100]}
gb_params = {'learning_rate': [0.1, 0.25, 0.5, 1, 5],
             'n_estimators': [10, 25, 50, 75, 100],
             'max_depth': [3, 5, 10, 15, 20],
             'min_samples_leaf': [5, 10, 20, 50, 100]}
xgb_params = {'learning_rate': [0.1, 0.25, 0.5, 1, 5],
             'n_estimators': [10, 25, 50, 75, 100],
             'max_depth': [3, 5, 10, 15, 20]}
knn_params = {'n_neighbors': [10, 20, 30, 40, 50],
              'leaf_size': [10, 20, 30, 40, 50]}

all_models = {'Decision Tree': [dt_model, dt_params], 'Random Forest': [rf_model, rf_params], 'Gradient Boost': [gb_model, gb_params],
               'XG Boost': [xgb_model, xgb_params], 'K Nearest Neighbours': [knn_model, knn_params]}

# define the folds to be used for cross validation
k_folds = KFold(n_splits=5, shuffle=True, random_state=0)

# Training Via Hypertuning
In this section, each model is trained using each combination of their parameters. <br />
For each algorithm, its highest scoring model (including its parameters) is stored, along with its score. <br />

Note: This training process can take a few minutes, as there are 925 models that are trained.

In [14]:
def evaluate_grid_search_cv(estimator, params, folds, X, y):
    grid_search = GridSearchCV(estimator=estimator, param_grid=params, cv=folds, n_jobs=-1, scoring='accuracy')
    grid_search.fit(X, y)
    best_model = grid_search.best_estimator_
    best_score = grid_search.best_score_
    return [best_model, best_score]

for key, items in all_models.items():
    print("Training " + key + " Model...")
    [model, params] = items[0], items[1]
    [best_model, best_score] = evaluate_grid_search_cv(model, params, k_folds, X, y)
    items.extend([str(best_model), best_score, best_model])
print("Training Complete!")

Training Decision Tree Model...
Training Random Forest Model...
Training Gradient Boost Model...
Training XG Boost Model...
Training K Nearest Neighbours Model...
Training Complete!


## Model Comparison
Below is a table containing each algorithm with its best performing model and its accuracy obtained. <br />
We see that our best scoring model is a Decision Tree with max_depth=3 and min_samples_leaf=10, with a score of 81.1%.

In [19]:
best_score_df = pd.DataFrame.from_dict(data=all_models, orient='index', 
                                        columns=['Model', 'Parameters', 'Best Model Label', 'Best Average Accuracy', 'Best Model'])
best_score_df.loc["Baseline Model"] = [str(baseline_model), None, str(baseline_model), baseline_score, baseline_model]
#print(tabulate(best_score_df[['Best Model Label', 'Best Average Accuracy']].sort_values(by='Best Average Accuracy', ascending=False),
#               headers='keys', tablefmt='fancy_grid'))
print(best_score_df[['Best Model Label', 'Best Average Accuracy']].sort_values(by='Best Average Accuracy', ascending=False))

                                                       Best Model Label  \
Decision Tree         DecisionTreeClassifier(max_depth=3, min_sample...   
XG Boost              XGBClassifier(base_score=0.5, booster='gbtree'...   
Gradient Boost        GradientBoostingClassifier(max_depth=15, min_s...   
Random Forest         RandomForestClassifier(max_depth=20, min_sampl...   
K Nearest Neighbours  KNeighborsClassifier(leaf_size=10, n_neighbors...   
Baseline Model                   DecisionTreeClassifier(random_state=0)   

                      Best Average Accuracy  
Decision Tree                         0.811  
XG Boost                              0.811  
Gradient Boost                        0.797  
Random Forest                         0.771  
K Nearest Neighbours                  0.754  
Baseline Model                        0.644  


## Estimating The Dollar Amount Paid To Fraudulent Claims
In this final section, we take a look at the dollar amount paid in all claims, actual fraud claims and predicted fraud claims.

In [16]:
# use the optimal model to predict whether each claim is fraudulent and append the predictions to our original data
optimal_row = best_score_df['Best Average Accuracy'].idxmax()
optimal_model = best_score_df.loc[optimal_row]['Best Model']
optimal_preds = optimal_model.predict(X)
raw_data['predicted_fraud'] = optimal_preds

# store the total amount of actual and predicted fraudulent claims
total_claim_amount = raw_data['total_claim_amount'].sum()
actual_fraud_amount = raw_data[raw_data['fraud_reported'] == 1]['total_claim_amount'].sum()
predicted_fraud_amount = raw_data[raw_data['predicted_fraud'] == 1]['total_claim_amount'].sum()
actual_fraud_percentage = round((actual_fraud_amount/total_claim_amount) * 100, 2)
predicted_fraud_percentage = round((predicted_fraud_amount/total_claim_amount) * 100, 2)

# display the amounts paid in all claims, actual fraud claims and predicted fraud claims
# and the difference between actual and predicted
print("Total claim amount: $" + str(raw_data['total_claim_amount'].sum()))
print("Total claim amount of actual fraudulent claims: $" + str(actual_fraud_amount) +
        ", " + str(actual_fraud_percentage) + "% of total claim amount")
print("Total claim amount of predicted fraudulent claims: $" + str(predicted_fraud_amount) +
        ", " + str(predicted_fraud_percentage) + "% of total claim amount")
print("Total difference between actual and predicted amounts: $" + str(abs(actual_fraud_amount - predicted_fraud_amount)) +
        " or " + str(round(abs(actual_fraud_percentage - predicted_fraud_percentage),2)) + "%")

Total claim amount: $52761940
Total claim amount of actual fraudulent claims: $14894620, 28.23% of total claim amount
Total claim amount of predicted fraudulent claims: $17682540, 33.51% of total claim amount
Total difference between actual and predicted amounts: $2787920 or 5.28%


Wrapping up, we see that although our optimal model had an accuracy of just 81.1%, it over estimates the dollar amount in fraudulent claims by $2787920, which is 5.28% more of the total claim amount than the actual amount. <br />

Not bad!